In [1]:
import pandas as pd

In [2]:
import mysql.connector as mc
from mysql.connector import Error
import mysql.connector

## Conexión SQL

In [7]:
# Conexión a SQL usando pymysql
def serverCon(host_name, user_name, password, db, pt):
    import pymysql
    connection = None
    try:
        connection = pymysql.connect(
            host=host_name,
            user=user_name,
            password=password,
            database=db,
            port=int(pt),
            charset='utf8mb4',
            cursorclass=pymysql.cursors.DictCursor,
            autocommit=True
        )
        print('Conexión exitosa')
    except Exception as err:
        print(f"Error al conectar: '{err}'")
    return connection

In [8]:
#Ejecución Query
def executeQuery(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print('Query Ejecutado')
    except Error as err:
        print(f"Error: '{err}'")

In [9]:
conect = serverCon('192.168.200.212', 'industrial', 'p@ss4DB', 'jornada_ii', '3306') #Puerto 3306

Conexión exitosa


In [14]:
# Listar todas las tablas de la base de datos
def get_table_names(connection):
    with connection.cursor() as cursor:
        cursor.execute("SHOW TABLES;")
        tables = [row[list(row.keys())[0]] for row in cursor.fetchall()]
    return tables

tablas = get_table_names(conect)
tablas

['actividades',
 'asistencias',
 'configuracion_sistema',
 'equipos_concurso',
 'estados_mexico',
 'inscripciones_workshop',
 'participantes',
 'v_asistencias_stats',
 'v_asistencias_workshops',
 'v_participantes_completos',
 'v_workshop_stats']

In [17]:
# Extraer y mostrar la tabla 'participantes' (sin duplicar encabezado)
df_participantes = pd.read_sql('SELECT * FROM participantes', conect)
print('Filas:', len(df_participantes))
display(df_participantes.head())
print('Columnas:', df_participantes.columns.tolist())

Filas: 236


C:\Users\20030\AppData\Local\Temp\ipykernel_16472\1691784127.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_participantes = pd.read_sql('SELECT * FROM participantes', conect)


,id,apellido_paterno,apellido_materno,primer_nombre,segundo_nombre,email,telefono,categoria,programa,brazalete,creado,actualizado
0,id,apellido_paterno,apellido_materno,primer_nombre,segundo_nombre,email,telefono,categoria,programa,brazalete,creado,actualizado
1,id,apellido_paterno,apellido_materno,primer_nombre,segundo_nombre,email,telefono,categoria,programa,brazalete,creado,actualizado
2,id,apellido_paterno,apellido_materno,primer_nombre,segundo_nombre,email,telefono,categoria,programa,brazalete,creado,actualizado
3,id,apellido_paterno,apellido_materno,primer_nombre,segundo_nombre,email,telefono,categoria,programa,brazalete,creado,actualizado
4,id,apellido_paterno,apellido_materno,primer_nombre,segundo_nombre,email,telefono,categoria,programa,brazalete,creado,actualizado


Columnas: ['id', 'apellido_paterno', 'apellido_materno', 'primer_nombre', 'segundo_nombre', 'email', 'telefono', 'categoria', 'programa', 'brazalete', 'creado', 'actualizado']


In [ ]:
# Descargar el DataFrame de participantes como CSV
df_participantes.to_csv('participantes.csv', index=False)
print('Archivo participantes.csv guardado.')

In [19]:
# Gráficos rápidos con pandas y plotly
import plotly.express as px
if not df_participantes.empty:
    # Participantes por programa
    fig = px.histogram(df_participantes, x='programa', title='Participantes por Programa')
    fig.show()
    # Participantes por dominio de correo
    df_participantes['dominio'] = df_participantes['email'].str.split('@').str[-1]
    fig2 = px.histogram(df_participantes, x='dominio', title='Participantes por Dominio de Correo')
    fig2.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

## Dashboard básico para participantes